In [1]:
import pandas as pd
import numpy as np
from implicit.als import AlternatingLeastSquares
from tqdm import tqdm_notebook
import scipy.sparse as sparse

In [1]:
#Загружаем данные

In [2]:
bookmark_tags=pd.read_csv('./hetrec2011-delicious-2k/bookmark_tags.dat', sep='\t')

In [90]:
bookmarks=pd.read_csv('./hetrec2011-delicious-2k/bookmarks.dat', sep='\t', encoding='Latin-1')

In [4]:
user_contacts=pd.read_csv('./hetrec2011-delicious-2k/user_contacts.dat', sep='\t')

In [5]:
user_contacts_timestamps=pd.read_csv('./hetrec2011-delicious-2k/user_contacts-timestamps.dat', sep='\t')

In [6]:
user_taggedbookmarks=pd.read_csv('./hetrec2011-delicious-2k/user_taggedbookmarks.dat', sep='\t')

In [7]:
user_taggedbookmarks_timestamps=pd.read_csv('./hetrec2011-delicious-2k/user_taggedbookmarks-timestamps.dat', sep='\t')

In [8]:
bookmark_tags.head()

,bookmarkID,tagID,tagWeight
0,1,2,276
1,1,206,48
2,1,208,165
3,1,272,55
4,1,459,67


In [9]:
bookmarks.head()

,id,md5,title,url,md5Principal,urlPrincipal
0,1,ab4954b633ddaf5b5bba6e9b71aa6b70,IFLA - The official website of the Internation...,http://www.ifla.org/,7f431306c428457bc4e12b15634484f,www.ifla.org
1,2,2221e9cd106d269dd34682666f576fa3,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf,1ef8cfcfe968101fa9b4e301847503d4,archive.ifla.org
2,7,c97c571dadaddbbb493126a0d4d01ba3,EdSelect,http://www.edselect.com/,792fd7eb20143386d0c4eb193c6124d,www.edselect.com
3,8,25bfe8dca0ef263ec9c341b9f16c38b5,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...,6fce4f6391516f0732531d9cfacda5b7,www.collectionscanada.gc.ca
4,9,c97284629e17b8e2861afaacd59918bc,Kidsreads.com,http://www.kidsreads.com/,5854ce8404857a45373eea01a3d98000,www.kidsreads.com


In [10]:
user_contacts.head()

,userID,contactID,date_day,date_month,date_year,date_hour,date_minute,date_second
0,8,28371,4,10,2010,2,14,19
1,8,40306,3,10,2010,12,33,9
2,8,85279,2,10,2010,3,10,48
3,8,91764,4,10,2010,22,43,47
4,8,97723,12,10,2010,3,28,33


In [11]:
user_contacts_timestamps.head()

,userID,contactID,timestamp
0,8,28371,1286151259000
1,8,40306,1286101989000
2,8,85279,1285981848000
3,8,91764,1286225027000
4,8,97723,1286846913000


In [12]:
user_taggedbookmarks.head()

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
0,8,1,1,8,11,2010,23,29,22
1,8,2,1,8,11,2010,23,25,59
2,8,7,1,8,11,2010,18,55,1
3,8,7,6,8,11,2010,18,55,1
4,8,7,7,8,11,2010,18,55,1


In [13]:
user_taggedbookmarks_timestamps.head()

,userID,bookmarkID,tagID,timestamp
0,8,1,1,1289255362000
1,8,2,1,1289255159000
2,8,7,1,1289238901000
3,8,7,6,1289238901000
4,8,7,7,1289238901000


In [14]:
bookmarks['urlPrincipal'].value_counts()

www.youtube.com                   1048
www.nytimes.com                    639
en.wikipedia.org                   585
www.guardian.co.uk                 484
mashable.com                       449
code.google.com                    417
www.slideshare.net                 337
github.com                         325
d.hatena.ne.jp                     293
www.bbc.co.uk                      271
vimeo.com                          262
lifehacker.com                     261
www.readwriteweb.com               241
www.wired.com                      237
www.smashingmagazine.com           228
www.flickr.com                     224
www.google.com                     209
techcrunch.com                     155
news.bbc.co.uk                     129
sites.google.com                   123
wordpress.org                      112
www.pbs.org                        111
online.wsj.com                     110
www.ted.com                        110
sixrevisions.com                   102
www.makeuseof.com        

In [15]:
bookmarks['id'].describe()

count     69226.000000
mean      52539.897423
std       31259.962726
min           1.000000
25%       25186.250000
50%       52042.500000
75%       79822.750000
max      107253.000000
Name: id, dtype: float64

In [16]:
bookmarks.shape

(69226, 6)

In [17]:
bookmarks['id'].value_counts()

2047      1
23134     1
43648     1
88702     1
84604     1
94843     1
92792     1
72310     1
78451     1
74353     1
103015    1
105062    1
98917     1
100964    1
107105    1
21087     1
16989     1
47746     1
19036     1
29275     1
31322     1
25177     1
4695      1
6742      1
14930     1
10832     1
53839     1
55886     1
49741     1
51788     1
         ..
40377     1
38328     1
60855     1
52659     1
50610     1
56753     1
54704     1
9646      1
15789     1
13740     1
1450      1
5544      1
28071     1
26022     1
30116     1
23969     1
105881    1
103832    1
77199     1
81293     1
79244     1
69003     1
66954     1
71048     1
93575     1
91526     1
85379     1
89473     1
87424     1
2049      1
Name: id, Length: 69226, dtype: int64

In [18]:
bookmarks.sort_values('id', ascending=False).head(10)

,id,md5,title,url,md5Principal,urlPrincipal
69225,107253,95ed332ae5fb43861b4d2461739b1297,International Reading Association,http://www.ira.org/,3508841d603e569727ba2cc74b6d7d3e,www.ira.org
69224,107252,a56525a58819bccdb94f5a5bab138a8c,ReadWriteThink: Learning Beyond the Classroom,http://www.readwritethink.org/beyondtheclassro...,a59dd4579a44c144cccf0c981df68eef,www.readwritethink.org
69223,107251,2d8a315a9a30612a7499ba6f5d247999,Reading/Language Arts Sunshine State Standards,http://etc.usf.edu/flstandards/la/index.html,6f029cc4ea200ae73f8e1e8a2282b145,etc.usf.edu
69222,107248,91e862687e1955e6696f78c1a2695b3f,The Lexile Framework for Reading,http://www.lexile.com/DesktopDefault.aspx?view...,f9631d79252c5e94173fd89fe7d50a2e,www.lexile.com
69221,107247,5a76f2c82314ed0f8f89edfcc38ba0b8,Reader&#039;s Theater Editions (Readers Theatr...,http://www.aaronshep.com/rt/RTE.html,178bd039b83bec995b3d9cf6a77ae630,www.aaronshep.com
69220,107246,8564791e3afb1b80ada0cf09fb991aa9,Latin Dictionary,http://www.nd.edu/~archives/latin.htm,55c67e797e5bbd09f2c10bcf6bcebca5,www.nd.edu
69219,107242,3f5f28a04205ce4bf6e2ef7ce52a69ba,ReadWriteThink: Student Materials,http://readwritethink.org/student_mat/index.asp,a13755d74a8d180214b65fd138803a43,readwritethink.org
69218,107241,c61029a13d36f84fa9857ca38a2778f6,100 Best Books,http://www.teachersfirst.com/100books.cfm,77e098f854695d182c3572b741934f1d,www.teachersfirst.com
69217,107240,93253d0f316f7c1be1c928e4b5da8453,"Vocabulary, Free Word Puzzles and Activities",http://www.vocabulary.com/,e2e35fd35bb84de38427deb6dcb59472,www.vocabulary.com
69216,107238,dcf35d33eee229d23ec50209d8bda08b,Reading A-Z: The online guided reading program...,http://www.readinga-z.com/,7b10163e37f46f46659256e731039650,www.readinga-z.com


In [19]:
user_taggedbookmarks.sort_values('bookmarkID', ascending=False).head(10)

,userID,bookmarkID,tagID,day,month,year,hour,minute,second
351398,80270,107253,23,23,4,2008,14,22,31
351397,80270,107252,23,28,4,2008,12,30,34
351396,80270,107251,23,13,5,2008,21,30,12
351395,80270,107248,1970,24,6,2008,17,6,12
351394,80270,107248,158,24,6,2008,17,6,12
351393,80270,107248,23,24,6,2008,17,6,12
351392,80270,107247,26106,24,6,2008,17,18,38
351391,80270,107247,20305,24,6,2008,17,18,38
351390,80270,107247,2457,24,6,2008,17,18,38
351389,80270,107246,8459,6,7,2008,21,9,26


In [20]:
user_contacts.sort_values('contactID', ascending=False).head(10)

,userID,contactID,date_day,date_month,date_year,date_hour,date_minute,date_second
12911,90433,108035,7,7,2007,16,49,34
6088,43502,108035,6,7,2007,6,1,17
10308,70991,108035,22,8,2008,5,8,21
2416,17397,108035,22,8,2008,5,8,52
12148,84469,108035,2,5,2010,23,34,26
2433,17430,108035,2,5,2010,23,29,51
6840,48054,108035,22,4,2007,21,30,59
3740,28049,108035,29,7,2007,1,11,10
11144,77247,108035,2,5,2010,23,30,4
732,5108,108035,15,6,2008,17,55,5


In [65]:
data=bookmark_tags

In [91]:
bookmarks.head()

,id,md5,title,url,md5Principal,urlPrincipal
0,1,ab4954b633ddaf5b5bba6e9b71aa6b70,IFLA - The official website of the Internation...,http://www.ifla.org/,7f431306c428457bc4e12b15634484f,www.ifla.org
1,2,2221e9cd106d269dd34682666f576fa3,gcdp-e.pdf (application/pdf Object),http://archive.ifla.org/VII/s14/nd1/gcdp-e.pdf,1ef8cfcfe968101fa9b4e301847503d4,archive.ifla.org
2,7,c97c571dadaddbbb493126a0d4d01ba3,EdSelect,http://www.edselect.com/,792fd7eb20143386d0c4eb193c6124d,www.edselect.com
3,8,25bfe8dca0ef263ec9c341b9f16c38b5,Cool Canada (Collections Canada),http://www.collectionscanada.gc.ca/cool/index-...,6fce4f6391516f0732531d9cfacda5b7,www.collectionscanada.gc.ca
4,9,c97284629e17b8e2861afaacd59918bc,Kidsreads.com,http://www.kidsreads.com/,5854ce8404857a45373eea01a3d98000,www.kidsreads.com


In [ ]:
#Сделаем словарь id к title

In [95]:
bookmarks_title=bookmarks[['id','title']].drop_duplicates()
bookmarks_title['id'] = bookmarks_title.id.astype(str)

In [96]:
bookmarks_title.head()

,id,title
0,1,IFLA - The official website of the Internation...
1,2,gcdp-e.pdf (application/pdf Object)
2,7,EdSelect
3,8,Cool Canada (Collections Canada)
4,9,Kidsreads.com


In [97]:
bookmarks_id_title = {}

for index, row in tqdm_notebook(bookmarks_title.iterrows()):
    bookmarks_id_title[row.id] = row.title

In [100]:
bookmarks_id_title

{'1': 'IFLA - The official website of the International Federation of Library Associations and Institutions',
 '2': 'gcdp-e.pdf (application/pdf Object)',
 '7': 'EdSelect',
 '8': 'Cool Canada (Collections Canada)',
 '9': 'Kidsreads.com',
 '10': 'Kidsreads.com -- GREAT BOOKS FOR BOYS',
 '11': 'Moodle ::|:: SDUSD Educational Technology',
 '14': 'CountryReports.org â\x80\x94 Country Information',
 '15': 'Some of the Best Books, K-12',
 '16': 'Education World Â®: Lesson Plan: Lessons from the Library',
 '17': 'FREE -- Teaching Resources and Lesson Plans from the Federal Government',
 '18': 'CyberBee',
 '19': 'IXL Math',
 '20': 'Carol Hurst&#039;s Children&#039;s Literature Site - Reviews and teaching ideas for kids&#039; books.',
 '22': 'Center for Children&#039;s Books, University of Illinois',
 '24': 'Bound To Stay Bound Books, Inc. - Bookstore',
 '25': 'My Home Library',
 '26': 'Read, Click and Win with BookAdventure!',
 '27': 'Digital, Online and Traditional Storytelling Multimedia Pub

In [67]:
data.head()

,bookmarkID,tagID,tagWeight
0,1,2,276
1,1,206,48
2,1,208,165
3,1,272,55
4,1,459,67


In [68]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 487131 entries, 0 to 487130
Data columns (total 3 columns):
bookmarkID    487131 non-null int64
tagID         487131 non-null int64
tagWeight     487131 non-null int64
dtypes: int64(3)
memory usage: 11.1 MB


In [4]:
#делаем предобработку для создания спарс матрицы

In [116]:
tegs = list(np.sort(data.tagID.unique()))
bookmarks = list(np.sort(data.bookmarkID.unique()))

In [117]:
tegweights = list(data.tagWeight)

In [118]:
len(tegs)

14346

In [119]:
len(bookmarks)

68755

In [120]:
len(tegweights)

487131

In [121]:
rows = data.tagID.astype(int)
cols = data.bookmarkID.astype(int)

In [122]:
cols[0]

1

In [123]:
len(cols)

487131

In [124]:
len(rows)

487131

In [125]:
len(tegweights)

487131

In [126]:
data_sparse = sparse.csr_matrix((tegweights, (cols, rows)))

In [127]:
data_sparse

<107254x64250 sparse matrix of type '<class 'numpy.int32'>'
	with 487131 stored elements in Compressed Sparse Row format>

In [3]:
#обучаем ALS

In [112]:
model = AlternatingLeastSquares(factors=100,regularization=0.05, calculate_training_loss=True, iterations=100, num_threads=6)
model.fit(data_sparse)

In [2]:
#Получаем рекомендации по book_id

In [128]:
book_id = 1001

book_items = data_sparse.T.tocsr()
recommendations = model.recommend(book_id, book_items, N=10)

In [129]:
recommendations

[(7141, 0.5316148),
 (24062, 0.52977145),
 (5792, 0.50762254),
 (14444, 0.5062217),
 (50622, 0.5050863),
 (14096, 0.49316716),
 (29308, 0.49213994),
 (84595, 0.49015772),
 (29313, 0.48952866),
 (14428, 0.4890746)]

In [130]:
for r in recommendations:
    print(bookmarks_id_title[str(r[0])])

flexmonkey - Google Code
Cucumber - Making BDD fun
Continuous Integration
TestDriven.NET &gt; Home
Rhonabwy Â» Setting up a python CI server with Hudson
Jasmine: Javascript Testing Framework
Vows Â« Asynchronous BDD for Node
DanNorth.net Â» Whatâs in a Story?
Introduction to Test Driven Design (TDD)
Getting started with Autotest - Continuous Testing
